### Derived from 2022-01-26-PARP1-TL

### 01_Retrieve PARP1 from ChEMBL

In [ ]:
pip install chembl_webresource_client

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
import os
import numpy as np

In [ ]:
target = new_client.target
target_query = target.search('PARP1')
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

In [ ]:
ac = new_client.activity
raw = ac.filter(target_chembl_id=selected_target)
df0 = pd.DataFrame.from_dict(raw)
df0

In [ ]:
df0.to_csv('PARP1_00_bioactivity_data_raw.csv', index=False)

In [ ]:
res_IC50 = ac.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df_IC50 = pd.DataFrame.from_dict(res_IC50)

In [ ]:
df_IC50.to_csv('PARP1_01_bioactivity_IC50_data_raw.csv', index=False)

In [ ]:
#Remove missing data
df2_IC50 = df_IC50[df_IC50.standard_value.notna()]
df2_IC50 = df2_IC50[df_IC50.canonical_smiles.notna()]

In [ ]:
#Remove duplicate data in canonical_smiles column
df2_nr_IC50 = df2_IC50.drop_duplicates(['canonical_smiles'])

In [ ]:
#Selecting 3 column for further analysis
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3_IC50 = df2_nr_IC50[selection]

In [ ]:
df3_IC50.to_csv('PARP1_02_bioactivity_IC50_data_preprocessed.csv', index=False)

In [ ]:
df4_IC50 = pd.read_csv('PARP1_02_bioactivity_IC50_data_preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4_IC50.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")
    
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5_IC50 = pd.concat([df4_IC50, bioactivity_class], axis=1)

In [ ]:
df5_IC50.to_csv('PARP1_03_bioactivity_IC50_data_curated.csv', index=False)

### 02_Exploratory analysis

In [ ]:
#run with python 3.7
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
df = pd.read_csv('PARP1_03_bioactivity_IC50_data_curated.csv')

In [ ]:
df

In [ ]:
class_ac = df['class'].value_counts()
class_ac.columns = ['class','count']
class_table = pd.DataFrame(class_ac)
class_table

In [ ]:
class_table.to_csv('PARP1_bioactivity_profile_IC50.csv')

In [ ]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [ ]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [ ]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)
df_lipinski

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)
df_combined

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [ ]:
df_norm = norm_value(df_combined)
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.pIC50.describe()

In [ ]:
df_final.to_csv('PARP1_04_bioactivity_data_3class_pIC50.csv')

In [ ]:
#Removing the 'intermediate' bioactivity class
df_2class = df_final[df_final['class']  != 'intermediate']
df_2class

In [ ]:
df_2class.to_csv('PARP1_05_bioactivity_data_2class_pIC50.csv')

In [ ]:
#Statistical
mean_MW = df_2class.groupby('class')["MW"].mean()
SD_MW = df_2class.groupby('class')["MW"].std()
mean_LogP = df_2class.groupby('class')["LogP"].mean()
SD_LogP = df_2class.groupby('class')["LogP"].std()
mean_NumHDonors = df_2class.groupby('class')["NumHDonors"].mean()
SD_NumHDonors = df_2class.groupby('class')["NumHDonors"].std()
mean_NumHAcceptors = df_2class.groupby('class')["NumHAcceptors"].mean()
SD_NumHAcceptors = df_2class.groupby('class')["NumHAcceptors"].std()

In [ ]:
total = [mean_MW,SD_MW,mean_LogP,SD_LogP,mean_NumHDonors,SD_NumHDonors,mean_NumHAcceptors,SD_NumHAcceptors]
label=['meanMWac', 'meanMWin','SDMWac', 'SDMWin',
     'meanLogPac', 'meanLogPin','SDLogPac','SDLogPin',
     'meanNumHDonorsac','meanNumHDonorsin','SDNumHDonorsac','SDNumHDonorsin',
    'meanNumHAcceptorsac', 'meanNumHAcceptorsin','SDNumHAcceptorsac','SDNumHAcceptorsin']

In [ ]:
statistic_Ro5 = pd.concat(total, axis=0)
Ro5 = pd.DataFrame(statistic_Ro5, columns=["stat"])
Ro5['label']= label
Ro5

In [ ]:
Ro5.to_csv("Ro5_stat.csv",index=True)

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(7, 7))

sns.countplot(x='class', data=df_2class, edgecolor='black')

plt.xlabel('bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.tiff', bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize=(7, 7))

sns.scatterplot(x='MW', y='LogP', data=df_2class, hue='class', size='pIC50', edgecolor='black', alpha=0.2)
sns.set_style("whitegrid", {"ytick.major.size": 100,"xtick.major.size": 2, 'grid.linestyle': 'solid'})

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

plt.savefig('plot_MW_vs_LogP.tiff', bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'pIC50', data = df_2class, medianprops=dict(color="black", alpha=1, linewidth=2))

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_IC50.tiff')

####  Statistical analysis | Mann-Whitney U Test

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'class']
  df = df_2class[selection]
  active = df[df['class'] == 'active']
  active = active[descriptor]

  selection = [descriptor, 'class']
  df = df_2class[selection]
  inactive = df[df['class'] == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'
  
  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

In [ ]:
pip install statannotations

In [ ]:
pip install statsmodels

In [ ]:
from statannotations.Annotator import Annotator

#### MW

In [ ]:
plt.figure(figsize=(5.5, 5.5))

ax = sns.boxplot(x = 'class', y = 'MW', data = df_2class, showmeans = True, meanprops={"marker":"o","markerfacecolor":"white", 
                       "markeredgecolor":"black","markersize":"8"},
                     medianprops=dict(color="black", alpha=1, linewidth=3))
ax.axhline(500, ls='--',c = 'black')
ax.set(ylim=(100, 850))
ax.set(xlabel=None)

plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.tiff')

In [ ]:
mannwhitney('MW')

#### LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

ax = sns.boxplot(x = 'class', y = 'LogP', data = df_2class, showmeans = True, meanprops={"marker":"o","markerfacecolor":"white", 
                       "markeredgecolor":"black","markersize":"8"},
                     medianprops=dict(color="black", alpha=1, linewidth=3))
ax.axhline(5, ls='--',c = 'black')
ax.set(ylim=(-6, 12))
ax.set(xlabel=None)

plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.tiff')

In [ ]:
mannwhitney('LogP')

#### NumHDonor

In [ ]:
plt.figure(figsize=(5.5, 5.5))

ax = sns.boxplot(x = 'class', y = 'NumHDonors', data = df_2class, 
                 showmeans = True, meanprops={"marker":"o","markerfacecolor":"white", 
                       "markeredgecolor":"black","markersize":"8"},
                     medianprops=dict(color="black", alpha=1, linewidth=3))
ax.axhline(5, ls='--',c = 'black')
ax.set(ylim=(-0.5,10.5))
ax.set(xlabel=None)

plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')
               
plt.savefig('plot_NumHDonors.tiff')

In [ ]:
mannwhitney('NumHDonors')

#### NumHAcceptors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

ax = sns.boxplot(x = 'class', y = 'NumHAcceptors', data = df_2class, showmeans = True, meanprops={"marker":"o","markerfacecolor":"white", 
                       "markeredgecolor":"black","markersize":"8"},
                     medianprops=dict(color="black", alpha=1, linewidth=3))
ax.axhline(10, ls='--',c = 'black')
ax.set(ylim=(0, 20))
ax.set(xlabel=None)

plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.tiff')

In [ ]:
mannwhitney('NumHAcceptors')

In [ ]:
!ls

In [ ]:
!zip -r results.zip . -i *.csv *.tiff

### 03_Molecular descriptors

In [ ]:
pip install wget

In [ ]:
import wget
from zipfile import ZipFile
import pandas as pd

In [ ]:
#conda install m2-base

In [ ]:
#wget.download('https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/AtomPairs2DFingerprintCount.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/AtomPairs2DFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/EStateFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/ExtendedFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/Fingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/GraphOnlyFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/KlekotaRothFingerprintCount.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/KlekotaRothFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/MACCSFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/PubchemFingerprinter.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/SubstructureFingerprintCount.sh')
wget.download('https://raw.githubusercontent.com/tlerksuthirat/PaDEL/master/SubstructureFingerprinter.sh')

In [ ]:
zipObj = ZipFile("padel.zip", "r")
zipObj.extractall()
zipObj.close()

In [ ]:
df = pd.read_csv('PARP1_05_bioactivity_data_2class_pIC50.csv')
df

In [ ]:
df = df.sort_values(by='class',ascending=True)

In [ ]:
df.to_csv('PARP1_05_bioactivity_data_2class_pIC50_sort_class.csv', index=False)

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df_selection = df[selection]
df_selection.to_csv('molecule_pos_neg_sort.smi', sep='\t', index=False, header=False)

In [ ]:
df_selection

In [ ]:
!cat molecule_pos_neg_sort.smi | head -5

In [ ]:
!cat molecule_pos_neg_sort.smi | wc -l

In [ ]:
!cat AtomPairs2DFingerprintCount.sh
!bash AtomPairs2DFingerprintCount.sh

In [ ]:
!cat AtomPairs2DFingerprinter.sh
!bash AtomPairs2DFingerprinter.sh

In [ ]:
!cat EStateFingerprinter.sh
!bash EStateFingerprinter.sh

In [ ]:
!cat ExtendedFingerprinter.sh
!bash ExtendedFingerprinter.sh

In [ ]:
!cat Fingerprinter.sh
!bash Fingerprinter.sh

In [ ]:
!cat GraphOnlyFingerprinter.sh
!bash GraphOnlyFingerprinter.sh

In [ ]:
!cat KlekotaRothFingerprintCount.sh
!bash KlekotaRothFingerprintCount.sh

In [ ]:
!cat KlekotaRothFingerprinter.sh
!bash KlekotaRothFingerprinter.sh

In [ ]:
!cat MACCSFingerprinter.sh
!bash MACCSFingerprinter.sh

In [ ]:
!cat PubchemFingerprinter.sh
!bash PubchemFingerprinter.sh

In [ ]:
!cat SubstructureFingerprintCount.sh
!bash SubstructureFingerprintCount.sh

In [ ]:
!cat SubstructureFingerprinter.sh
!bash SubstructureFingerprinter.sh

### Read in fingerprints

In [ ]:
import os
import pandas as pd

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github')
df_x_AFC = pd.read_csv('descriptors_output_AtomPairs2DFingerprintCount.csv')
df_x_AF = pd.read_csv('descriptors_output_AtomPairs2DFingerprinter.csv')
df_x_ESF = pd.read_csv('descriptors_output_EStateFingerprinter.csv')
df_x_EXF = pd.read_csv('descriptors_output_ExtendedFingerprinter.csv')
df_x_F = pd.read_csv('descriptors_output_Fingerprinter.csv')
df_x_GF = pd.read_csv('descriptors_output_GraphOnlyFingerprinter.csv')
df_x_KRFC = pd.read_csv('descriptors_output_KlekotaRothFingerprintCount.csv')
df_x_KRF = pd.read_csv('descriptors_output_KlekotaRothFingerprinter.csv')
df_x_MF = pd.read_csv('descriptors_output_MACCSFingerprinter.csv')
df_x_PF = pd.read_csv('descriptors_output_PubchemFingerprinter.csv')
df_x_SFC = pd.read_csv('descriptors_output_SubstructureFingerprintCount.csv')
df_x_SF = pd.read_csv('descriptors_output_SubstructureFingerprinter.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('PARP1_05_bioactivity_data_2class_pIC50_sort_class.csv')
df =  df.iloc[:, 1:]
df

In [ ]:
os.mkdir('fingerprint')

In [ ]:
os.chdir('fingerprint')

In [ ]:
df_x_AFC.to_csv('AtomPairs2DCount.csv', index=False)
df_x_AF.to_csv('AtomPairs2D.csv', index=False)
df_x_ESF.to_csv('EState.csv', index=False)
df_x_EXF.to_csv('CDKextended.csv', index=False)
df_x_F.to_csv('CDK.csv', index=False)
df_x_GF.to_csv('CDKgraphonly.csv', index=False)
df_x_KRFC.to_csv('KlekotaRothCount.csv', index=False)
df_x_KRF.to_csv('KlekotaRoth.csv', index=False)
df_x_MF.to_csv('MACCS.csv', index=False)
df_x_PF.to_csv('PubChem.csv', index=False)
df_x_SFC.to_csv('SubstructureCount.csv', index=False)
df_x_SF.to_csv('Substructure.csv', index=False)

In [ ]:
# Create variables to house fingerprint descriptors as DataFrames
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github')

FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

for i in FP_list:
  fp = f'fingerprint/{i}.csv'
  descriptors = pd.read_csv(fp, index_col=False)
  exec(i + '= descriptors')

### 04_Model construction

In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
pip install scikit-learn

In [ ]:
# Data processing
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

# Import Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Cross-validation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

# Model performance metric
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef

In [ ]:
classifier_list = {'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
                  'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
                  'KNeighborsClassifier': KNeighborsClassifier(),
                  'MLPClassifier': MLPClassifier(random_state=42),
                  'RandomForestClassifier': RandomForestClassifier(random_state=42),
                  'SVC': SVC(kernel='rbf', random_state=42),#remove class_weight
                  'XGBClassifier': XGBClassifier(random_state=42),
                  'LGBMClassifier': LGBMClassifier(random_state=42),
                  'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42),
                  'GaussianProcessClassifier': GaussianProcessClassifier(random_state=42),
                  'GaussianNB': GaussianNB(),
                  'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis()}

In [ ]:
! pip install -U imbalanced-learn

In [ ]:
# Class balancing
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import pickle

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

def model_building(features_df, bioactivity_class, classifier, balancing):
  
  # Preparing X and y
  df = eval(features_df)
  X = df.drop('Name', axis=1)
  y = bioactivity_class
  y = pd.Series(y)

  def target_encode(val):
    target_mapper = {'inactive':0, 'active':1}
    return target_mapper[val]

  y = y.apply(target_encode)

  # Remove low variance features
  X2 = remove_low_variance(X, threshold=0.01)
  X2.to_csv(f'{balancing}_{features_df}_removed_low_variance_0.1.csv', index=False)
  
  # Data splitting
  X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)
  X_train.to_csv(f'{balancing}_{features_df}_X_train.csv', index=False)
  y_train.to_csv(f'{balancing}_{features_df}_y_train.csv', index=False)
  X_test.to_csv(f'{balancing}_{features_df}_X_test.csv', index=False)
  y_test.to_csv(f'{balancing}_{features_df}_y_test.csv', index=False)

  # Oversampling
  if balancing == 'oversampling':
    ros = RandomOverSampler(sampling_strategy="not majority") # String
    X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
    X_train = X_train_ros
    y_train = y_train_ros
  if balancing == 'undersampling':
    rus = RandomUnderSampler(sampling_strategy=1) # Numerical value
    X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)
    X_train = X_train_rus
    y_train = y_train_rus
  else:
    pass

  # Model building
  model = classifier_list[classifier]
  model.fit(X_train, y_train)
  # Saving the model
  pickle.dump(model, open(f'{balancing}_{features_df}_{classifier}.pkl', 'wb'))

  # Apply model to make prediction
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)
  
  # Building a CV model
  model_cv = classifier_list[classifier]
  #cv = cross_validate(model_cv, X_train, y_train, cv=5, scoring=make_scorer(matthews_corrcoef))
  cv_scoring = {'Ac': 'accuracy', 'Sn': make_scorer(recall_score), 'Sp': make_scorer(recall_score, pos_label=0), 'MCC': make_scorer(matthews_corrcoef)}
  cv = cross_validate(model_cv, X_train, y_train, cv=5, scoring=cv_scoring)

  # Calculating model performance
  ac_train = accuracy_score(y_train, y_train_pred)
  ac_test = accuracy_score(y_test, y_test_pred)
  ac_cv = cv['test_Ac'].mean()

  sn_train = recall_score(y_train, y_train_pred)
  sn_test = recall_score(y_test, y_test_pred)
  sn_cv = cv['test_Sn'].mean()

  sp_train = recall_score(y_train, y_train_pred, pos_label=0)
  sp_test = recall_score(y_test, y_test_pred, pos_label=0)
  sp_cv = cv['test_Sp'].mean()

  mcc_train = matthews_corrcoef(y_train, y_train_pred)
  mcc_test = matthews_corrcoef(y_test, y_test_pred)
  mcc_cv = cv['test_MCC'].mean()
  
  # Preparing performance summary table
  model_name = pd.Series([classifier], name='Algorithm')

  ac_train_series = pd.Series(ac_train, name='Ac_train')
  ac_test_series = pd.Series(ac_test, name='Ac_test')
  ac_cv_series = pd.Series(ac_cv, name='Ac_cv')

  sn_train_series = pd.Series(sn_train, name='Sn_train')
  sn_test_series = pd.Series(sn_test, name='Sn_test')
  sn_cv_series = pd.Series(sn_cv, name='Sn_cv')

  sp_train_series = pd.Series(sp_train, name='Sp_train')
  sp_test_series = pd.Series(sp_test, name='Sp_test')
  sp_cv_series = pd.Series(sp_cv, name='Sp_cv')

  mcc_train_series = pd.Series(mcc_train, name='MCC_train')
  mcc_cv_series = pd.Series(mcc_cv, name='MCC_cv')
  mcc_test_series = pd.Series(mcc_test, name='MCC_test')

#change the code -- recheck
  performance_metrics = pd.concat([model_name,
                                   ac_train_series, sn_train_series, sp_train_series, mcc_train_series,
                                   ac_cv_series, sn_cv_series, sp_cv_series,  mcc_cv_series,
                                   ac_test_series, sn_test_series, sp_test_series, mcc_test_series], axis=1)                           
  performance_metrics['MCC_train_cv'] = abs(performance_metrics['MCC_train'] - performance_metrics['MCC_cv'])
  performance_metrics['MCC_train_test'] = abs(performance_metrics['MCC_train'] - performance_metrics['MCC_test'])

  return performance_metrics

In [ ]:
# Iterating through both the Classifier and Fingerprint lists
df_list = []

for i in classifier_list:
  for j in FP_list:
    print(j, i)
    classifier = model_building(j, df['class'], i, 'normal')
    classifier['Fingerprint'] = j
    # Reorder last column to be first column
    cols = classifier.columns.tolist()
    cols = cols[-1:] + cols[:-1] 
    classifier = classifier[cols]
    # Append DataFrame to list
    df_list.append(classifier)

# Combine DataFrame from list
df_final_normal = pd.concat(df_list)
df_final_normal.sort_values(by=['Fingerprint'], inplace=True)

In [ ]:
# Iterating through both the Classifier and Fingerprint lists
df_list = []

for i in classifier_list:
  for j in FP_list:
    print(j, i)
    classifier = model_building(j, df['class'], i, 'oversampling')
    classifier['Fingerprint'] = j
    # Reorder last column to be first column
    cols = classifier.columns.tolist()
    cols = cols[-1:] + cols[:-1] 
    classifier = classifier[cols]
    # Append DataFrame to list
    df_list.append(classifier)

# Combine DataFrame from list
df_final_oversampling = pd.concat(df_list)
df_final_oversampling.sort_values(by=['Fingerprint'], inplace=True)

In [ ]:
# Iterating through both the Classifier and Fingerprint lists
df_list = []

for i in classifier_list:
  for j in FP_list:
    print(j, i)
    classifier = model_building(j, df['class'], i, 'undersampling')
    classifier['Fingerprint'] = j
    # Reorder last column to be first column
    cols = classifier.columns.tolist()
    cols = cols[-1:] + cols[:-1] 
    classifier = classifier[cols]
    # Append DataFrame to list
    df_list.append(classifier)

# Combine DataFrame from list
df_final_undersampling = pd.concat(df_list)
df_final_undersampling.sort_values(by=['Fingerprint'], inplace=True)

In [ ]:
#imbalanced data
df_final_normal.to_csv('normal_results.csv', index=False)

In [ ]:
df_final_oversampling.to_csv('oversampling_results.csv', index=False)

In [ ]:
df_final_undersampling.to_csv('undersampling_results.csv', index=False)

In [ ]:
! zip PARP1_normal_models_pkl.zip normal*.pkl
! zip PARP1_normal_lowvarianceremoved_datasplit.zip normal*.csv
! zip PARP1_oversampling_models_pkl.zip oversampling*.pkl
! zip PARP1_oversampling_lowvarianceremoved_datasplit.zip oversampling*.csv
! zip PARP1_undersampling_models_pkl.zip undersampling*.pkl
! zip PARP1_undersampling_lowvarianceremoved_datasplit.zip undersampling*.csv

In [ ]:
! mkdir ZIP
! mv *.zip ZIP

In [ ]:
! mkdir results
! mv *results.csv results

In [ ]:
! rm normal*.pkl normal*.csv oversampling*.pkl oversampling*.csv undersampling*.pkl undersampling*.csv

### 05_Post-Model Analysis
#### restart kernel to clear previous outputs

In [ ]:
import os
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')

In [ ]:
from zipfile import ZipFile
file_name = "PARP1_normal_lowvarianceremoved_datasplit.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting specific file
    print('Extracting all the files now...')
    zip.extract('normal_results.csv')
    print('Done!')

In [ ]:
from zipfile import ZipFile
file_name = "PARP1_oversampling_lowvarianceremoved_datasplit.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting specific file
    print('Extracting all the files now...')
    zip.extract('oversampling_results.csv')
    print('Done!')

In [ ]:
from zipfile import ZipFile
file_name = "PARP1_undersampling_lowvarianceremoved_datasplit.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting specific file
    print('Extracting all the files now...')
    zip.extract('undersampling_results.csv')
    print('Done!')

In [ ]:
import pandas as pd

df = pd.read_csv('normal_results.csv')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df2 = df.copy()
df2

In [ ]:
# Custom function for making Heatmap

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def make_heatmap(score):
  # Shaping the data
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  # Making the Heatmap
  fig, ax = plt.subplots()
  im = ax.imshow(z, cmap='RdBu', vmin=0, vmax=1)
  #im = ax.imshow(z, cmap='cool')

  # Show all ticks
  ax.set_xticks(np.arange(len(x)))
  ax.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax.set_xticklabels(x)
  ax.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

  # Add colorbar
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im, cax=cax)

  fig.set_size_inches(9, 9)
  fig.tight_layout()
  #plt.savefig(f'{score}.pdf')
  plt.savefig(f'{score}.tiff')

In [ ]:
os.mkdir('normal_results')

#### MCC for Training set

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP\normal_results')
make_heatmap('MCC_train')

#### MCC for test set

In [ ]:
make_heatmap('MCC_test')

#### MCC for CV set

In [ ]:
make_heatmap('MCC_cv')

In [ ]:
# Custom function for making Heatmap

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

def make_heatmap_panel_plot():
  # Shaping the data

  # Making the Heatmap
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
  #im = ax.imshow(z, cmap='cool')

# ax1
  # Data
  score1 = 'MCC_train'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score1]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score1]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im1 = ax1.imshow(z, cmap='RdBu', vmin=0, vmax=1)
  
  #add color bar
  divider = make_axes_locatable(ax1)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im1, cax=cax)

  # Show all ticks
  ax1.set_xticks(np.arange(len(x)))
  ax1.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax1.set_xticklabels(x)
  ax1.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax1.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax1.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax2
  # Data
  score2 = 'MCC_cv'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score2]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score2]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im2 = ax2.imshow(z, cmap='RdBu', vmin=0, vmax=1)
    
  #add color bar
  divider = make_axes_locatable(ax2)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im2, cax=cax)

  # Show all ticks
  ax2.set_xticks(np.arange(len(x)))
  ax2.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax2.set_xticklabels(x)
  ax2.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax2.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax2.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax3
  # Data
  score3 = 'MCC_test'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score3]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score3]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im3 = ax3.imshow(z, cmap='RdBu', vmin=0, vmax=1)

  #add color bar
  divider = make_axes_locatable(ax3)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im3, cax=cax)

  # Show all ticks
  ax3.set_xticks(np.arange(len(x)))
  ax3.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax3.set_xticklabels(x)
  ax3.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax3.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax3.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")


  # Y tick labels
  ax2.yaxis.set_ticklabels([])
  ax3.yaxis.set_ticklabels([])

  # Sub-plot title
  ax1.set_title('Training set', fontsize=14, fontweight='bold', pad=15) # Training set
  ax2.set_title('CV set', fontsize=14, fontweight='bold', pad=15) # CV set
  ax3.set_title('Test set', fontsize=14, fontweight='bold', pad=15) # Test set

  # Axes labels
  ax1.set_ylabel('Fingerprints', fontweight='bold', fontsize=14, labelpad=15)
  ax2.set_xlabel('MCC', fontweight='bold', fontsize=14, labelpad=15)
     
  fig.set_size_inches(18, 10.5)
  fig.tight_layout()
  #plt.savefig('Fig_Heatmap_Model_Performance.pdf')
  plt.savefig('Fig_Heatmap_Model_Performance.tiff')

make_heatmap_panel_plot()

#### MCC of Training minus MCC of CV set

In [ ]:
make_heatmap('MCC_train_cv')

#### MCC of Training minus MCC of Test set

In [ ]:
make_heatmap('MCC_train_test')

#### Oversampling_results

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')
df = pd.read_csv('oversampling_results.csv')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df2 = df.copy()
df2

In [ ]:
os.mkdir('oversampling_results')

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP\oversampling_results')
make_heatmap('MCC_train')

In [ ]:
make_heatmap('MCC_test')

In [ ]:
make_heatmap('MCC_cv')

In [ ]:
# Custom function for making Heatmap

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

def make_heatmap_panel_plot():
  # Shaping the data

  # Making the Heatmap
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
  #im = ax.imshow(z, cmap='cool')

# ax1
  # Data
  score1 = 'MCC_train'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score1]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score1]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im1 = ax1.imshow(z, cmap='RdBu', vmin=0, vmax=1)
  
  #add color bar
  divider = make_axes_locatable(ax1)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im1, cax=cax)

  # Show all ticks
  ax1.set_xticks(np.arange(len(x)))
  ax1.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax1.set_xticklabels(x)
  ax1.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax1.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax1.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax2
  # Data
  score2 = 'MCC_cv'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score2]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score2]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im2 = ax2.imshow(z, cmap='RdBu', vmin=0, vmax=1)
    
  #add color bar
  divider = make_axes_locatable(ax2)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im2, cax=cax)

  # Show all ticks
  ax2.set_xticks(np.arange(len(x)))
  ax2.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax2.set_xticklabels(x)
  ax2.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax2.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax2.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax3
  # Data
  score3 = 'MCC_test'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score3]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score3]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im3 = ax3.imshow(z, cmap='RdBu', vmin=0, vmax=1)

  #add color bar
  divider = make_axes_locatable(ax3)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im3, cax=cax)

  # Show all ticks
  ax3.set_xticks(np.arange(len(x)))
  ax3.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax3.set_xticklabels(x)
  ax3.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax3.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax3.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")


  # Y tick labels
  ax2.yaxis.set_ticklabels([])
  ax3.yaxis.set_ticklabels([])

  # Sub-plot title
  ax1.set_title('Training set', fontsize=14, fontweight='bold', pad=15) # Training set
  ax2.set_title('CV set', fontsize=14, fontweight='bold', pad=15) # CV set
  ax3.set_title('Test set', fontsize=14, fontweight='bold', pad=15) # Test set

  # Axes labels
  ax1.set_ylabel('Fingerprints', fontweight='bold', fontsize=14, labelpad=15)
  ax2.set_xlabel('MCC', fontweight='bold', fontsize=14, labelpad=15)
     
  fig.set_size_inches(18, 10.5)
  fig.tight_layout()
  #plt.savefig('Fig_Heatmap_Model_Performance.pdf')
  plt.savefig('Fig_Heatmap_Model_Performance.tiff')

make_heatmap_panel_plot()

In [ ]:
make_heatmap('MCC_train_cv')

In [ ]:
make_heatmap('MCC_train_test')

#### Undersampling_results

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')
df = pd.read_csv('undersampling_results.csv')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df2 = df.copy()
df2

In [ ]:
os.mkdir('undersampling_results')

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP\undersampling_results')
make_heatmap('MCC_train')

In [ ]:
make_heatmap('MCC_test')

In [ ]:
make_heatmap('MCC_cv')

In [ ]:
# Custom function for making Heatmap

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

def make_heatmap_panel_plot():
  # Shaping the data

  # Making the Heatmap
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
  #im = ax.imshow(z, cmap='cool')

# ax1
  # Data
  score1 = 'MCC_train'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score1]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score1]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im1 = ax1.imshow(z, cmap='RdBu', vmin=0, vmax=1)
  
  #add color bar
  divider = make_axes_locatable(ax1)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im1, cax=cax)

  # Show all ticks
  ax1.set_xticks(np.arange(len(x)))
  ax1.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax1.set_xticklabels(x)
  ax1.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax1.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax1.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax2
  # Data
  score2 = 'MCC_cv'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score2]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score2]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im2 = ax2.imshow(z, cmap='RdBu', vmin=0, vmax=1)
    
  #add color bar
  divider = make_axes_locatable(ax2)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im2, cax=cax)

  # Show all ticks
  ax2.set_xticks(np.arange(len(x)))
  ax2.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax2.set_xticklabels(x)
  ax2.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax2.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax2.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")

# ax3
  # Data
  score3 = 'MCC_test'
  grid_results = pd.concat([df2.Fingerprint, df2.Algorithm, df2[score3]], axis=1)
  grid_contour = grid_results.groupby(['Fingerprint','Algorithm']).mean()
  grid_reset = grid_contour.reset_index()
  grid_reset.columns = ['Fingerprint', 'Algorithm', score3]
  grid_pivot = grid_reset.pivot('Fingerprint', 'Algorithm')
  x = grid_pivot.columns.levels[1].values
  y = grid_pivot.index.values
  z = np.round(grid_pivot.values, 2)

  im3 = ax3.imshow(z, cmap='RdBu', vmin=0, vmax=1)

  #add color bar
  divider = make_axes_locatable(ax3)
  cax = divider.append_axes("right", size="5%", pad=0.2)
  plt.colorbar(im3, cax=cax)

  # Show all ticks
  ax3.set_xticks(np.arange(len(x)))
  ax3.set_yticks(np.arange(len(y)))
  # Label all ticks with list entries
  ax3.set_xticklabels(x)
  ax3.set_yticklabels(y)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax3.get_xticklabels(), rotation=90, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(y)):
      for j in range(len(x)):
          text = ax3.text(j, i, z[i, j],
                        ha="center", va="center", color="white", fontweight="bold")


  # Y tick labels
  ax2.yaxis.set_ticklabels([])
  ax3.yaxis.set_ticklabels([])

  # Sub-plot title
  ax1.set_title('Training set', fontsize=14, fontweight='bold', pad=15) # Training set
  ax2.set_title('CV set', fontsize=14, fontweight='bold', pad=15) # CV set
  ax3.set_title('Test set', fontsize=14, fontweight='bold', pad=15) # Test set

  # Axes labels
  ax1.set_ylabel('Fingerprints', fontweight='bold', fontsize=14, labelpad=15)
  ax2.set_xlabel('MCC', fontweight='bold', fontsize=14, labelpad=15)
     
  fig.set_size_inches(18, 10.5)
  fig.tight_layout()
  #plt.savefig('Fig_Heatmap_Model_Performance.pdf')
  plt.savefig('Fig_Heatmap_Model_Performance.tiff')

make_heatmap_panel_plot()

In [ ]:
make_heatmap('MCC_train_cv')

In [ ]:
make_heatmap('MCC_train_test')

### 06_Feature importance

In [ ]:
import os
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')

#### Select pubchem

In [ ]:
from zipfile import ZipFile
file_name = "PARP1_oversampling_models_pkl.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting specific file
    print('Extracting all the files now...')
    zip.extract('oversampling_PubChem_RandomForestClassifier.pkl')
    print('Done!')

In [ ]:
from zipfile import ZipFile
file_name = "PARP1_oversampling_lowvarianceremoved_datasplit.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting specific file
    print('Extracting all the files now...')
    zip.extract('oversampling_PubChem_X_train.csv')
    zip.extract('oversampling_PubChem_y_train.csv')
    zip.extract('oversampling_PubChem_X_test.csv')
    zip.extract('oversampling_PubChem_y_test.csv')
    print('Done!')

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')

In [ ]:
import pandas as pd
import pickle

X_train = pd.read_csv('oversampling_PubChem_X_train.csv')
y_train = pd.read_csv('oversampling_PubChem_y_train.csv')
X_test = pd.read_csv('oversampling_PubChem_X_test.csv')
y_test = pd.read_csv('oversampling_PubChem_y_test.csv')

load_model = pickle.load(open('oversampling_PubChem_RandomForestClassifier.pkl', 'rb'))

X_train_pred = load_model.predict(X_train)

In [ ]:
os.mkdir('gini')

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP\gini')

In [ ]:
features = pd.concat([pd.Series(X_train.columns, name='Features'), pd.Series(load_model.feature_importances_, name='Gini')], axis=1 )
features.sort_values(by='Gini', ascending=False, inplace=True)
features

In [ ]:
features.to_csv('feature_Gini_oversampling_PubChem_RandomForestClassifier.csv')

In [ ]:
# Create feature importance plot
import matplotlib.pyplot as plt
plt.figure(figsize=(6,8))
plt.barh(features.Features, features.Gini, color='#7CAE00', edgecolor='black', align='center', alpha=0.8)
plt.ylim(-0.5,19.5)
plt.gca().invert_yaxis()
plt.grid(True)

plt.xlabel('Gini Index', fontsize=14, fontweight='bold', labelpad=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0, 0.025)

plt.margins(0.02)
plt.tight_layout()

plt.savefig('Barplot_feature_importance_oversampling_PubChem_RandomForestClassifier.tiff')

### 07_PCA analysis

In [ ]:
# PCA modeling
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

# Data scaling
X_train = scale(X_train)
X_test = scale(X_test)

# Build PCA model
pca = PCA()
pca.fit(X_train)
pca.fit(X_test)

In [ ]:
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP')
os.mkdir('PCA_scores')
os.chdir(r'C:\Users\tassa\Desktop\##Chemoinformatics\##Code for depositing in Github\ZIP\PCA_scores')

In [ ]:
# Screen plot
import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.savefig('Screen plot.tiff')

In [ ]:
# Compute PC scores train
scores_train = pca.transform(X_train)
scores_df_train = pd.DataFrame(scores_train)
scores_df_train

In [ ]:
# Compute PC scores test
scores_test = pca.transform(X_test)
scores_df_test = pd.DataFrame(scores_test)
scores_df_test

In [ ]:
scores_df_train.to_csv('scores_df_train.csv')
scores_df_test.to_csv('scores_df_test.csv')

In [ ]:
#oversee max min
train_max = scores_df_train.values.max()
train_min = scores_df_train.values.min()
test_max = scores_df_test.values.max()
test_min = scores_df_test.values.min()
print(train_max,train_min)
print(test_max,test_min)

In [ ]:
# PCA scores plot
import matplotlib.pyplot as plt

plt.figure(figsize=(25,25))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

# PC1 vs PC2
ax1.scatter(x=scores_df_train[0], y=scores_df_train[1], c="#7CAE00", alpha=0.2, edgecolor='black', label='train') #green
ax1.scatter(x=scores_df_test[0], y=scores_df_test[1], c="#f25555", alpha=0.2, edgecolor='black', label='test') #red

# PC1 vs PC3
ax2.scatter(x=scores_df_train[0], y=scores_df_train[2], c="#7CAE00", alpha=0.2, edgecolor='black', label='train')
ax2.scatter(x=scores_df_test[0], y=scores_df_test[2], c="#f25555", alpha=0.2, edgecolor='black', label='test')

# PC2 vs PC3
ax3.scatter(x=scores_df_train[1], y=scores_df_train[2], c="#7CAE00", alpha=0.2, edgecolor='black', label='train')
ax3.scatter(x=scores_df_test[1], y=scores_df_test[2], c="#f25555", alpha=0.2, edgecolor='black', label='test')

# X tick limits
ax1.set_xlim(-15, 20)
ax2.set_xlim(-15, 20)
ax3.set_xlim(-18, 20)

# Y tick labels
ax1.set_ylim(-18, 20)
ax2.set_ylim(-12, 35)
ax3.set_ylim(-12, 35)

# Axes labels
ax1.set_xlabel('PC1', fontsize=14, fontweight='bold')
ax1.set_ylabel('PC2', fontsize=14, fontweight='bold', labelpad = -2)
ax1.grid()

ax2.set_xlabel('PC1', fontsize=14, fontweight='bold')
ax2.set_ylabel('PC3', fontsize=14, fontweight='bold', labelpad = -2)
ax2.grid()

ax3.set_xlabel('PC2', fontsize=14, fontweight='bold')
ax3.set_ylabel('PC3', fontsize=14, fontweight='bold', labelpad = -2)
ax3.grid()

fig.set_size_inches(15, 5)

fig.tight_layout()

plt.savefig('PCA_scores.tiff')